In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = r'/reference notebook/data.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
# function to perform Min-Max scaling with bias
def min_max_scaling_with_bias(column, bias_min=None, bias_max=None):
    # calculate the minimum and maximum values of the column
    column_min = column.min()
    column_max = column.max()

    # add bias to the minimum and maximum values if specified
    if bias_min is not None:
        column_min -= bias_min
    if bias_max is not None:
        column_max += bias_max

    # perform Min-Max scaling
    column_scaled = (column - column_min) / (column_max - column_min)

    return column_scaled, column_min, column_max

In [ ]:
# function to perform inverse transformation and return the original values
def min_max_scaling_inverse(column_scaled, column_min, column_max, bias_min=None, bias_max=None):
    # remove bias from the minimum and maximum values if specified
    if bias_min is not None:
        column_min += bias_min
    if bias_max is not None:
        column_max -= bias_max

    # perform inverse transformation
    column_unscaled = column_scaled * (column_max - column_min) + column_min

    return column_unscaled

In [ ]:
scaled_open, min_val, max_val = min_max_scaling_with_bias(df['Open'],bias_min=1000, bias_max=1000)

In [ ]:
input_seq_len = 380
output_seq_len = 1
input_data = []
output_data = []

for i in range(len(scaled_open) - input_seq_len - output_seq_len):
    input_data.append(scaled_open[i:i+input_seq_len])
    output_data.append(scaled_open[i+input_seq_len:i+input_seq_len+output_seq_len])
input_data = np.array(input_data)
output_data = np.array(output_data)

In [ ]:
train_size = int(len(input_data) * 0.8)
x_train = input_data[:train_size]
y_train = output_data[:train_size]
x_test = input_data[train_size:]
y_test = output_data[train_size:]

# Print the shapes of the resulting data sets
print("Training input shape:", x_train.shape)
print("Training output shape:", y_train.shape)
print("Test input shape:", x_test.shape)
print("Test output shape:", y_test.shape)

In [ ]:
train_size = int(len(input_data) * 0.8)
x_train = input_data[:train_size]
y_train = output_data[:train_size]
x_test = input_data[train_size:]
y_test = output_data[train_size:]

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    # Input layer
    tf.keras.layers.Input(shape=(380,)),

    # First CNN layer
    tf.keras.layers.Reshape((10, 38, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Second CNN layer
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # LSTM layer
    tf.keras.layers.Reshape((1, 8 * 64)),
    tf.keras.layers.LSTM(32),

    # Output layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()
model.compile(loss='mse', optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              verbose=1)

# Train your model with the callback
model.fit(x_train, y_train, epochs=30, callbacks=[cp_callback])

In [ ]:
ypred = model.predict(x_train)
unscaled_pred = min_max_scaling_inverse(ypred, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)
len(unscaled_pred)

In [ ]:
import matplotlib.pyplot as plt
unscaled_act = min_max_scaling_inverse(y_train, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)

plt.plot(unscaled_pred[:100])
plt.plot(unscaled_act[:100])


In [ ]:
ypred = model.predict(x_test)
unscaled_pred = min_max_scaling_inverse(ypred, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)
unscaled_act = min_max_scaling_inverse(y_test, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)

plt.plot(unscaled_pred)
plt.plot(unscaled_act)

In [ ]:
plt.plot(unscaled_pred[:100])
plt.plot(unscaled_act[:100])

In [ ]:
model.save('Try 6.h5')

In [ ]:
from keras.models import load_model

model = load_model('Try 6.h5')
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              verbose=1)

# Train your model with the callback
model.fit(x_train, y_train, epochs=100, callbacks=[cp_callback])

In [ ]:
model.save('Try 61.h5')

In [ ]:
import matplotlib.pyplot as plt

ypred = model.predict(x_test)
unscaled_pred = min_max_scaling_inverse(ypred, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)
unscaled_act = min_max_scaling_inverse(y_test, min_val -1000, max_val + 1000, bias_min=1000, bias_max=1000)

plt.plot(unscaled_pred)
plt.plot(unscaled_act)

In [ ]:
plt.plot(unscaled_pred[0:100])
plt.plot(unscaled_act[0:100])

In [ ]:
y_test[0]